<a href="https://colab.research.google.com/github/joel280584/PineConeLearning/blob/main/Introduction_to_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Pinecone

In [1]:
!pip install pinecone
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
apikey = userdata.get('API_KEY')
from pinecone import Pinecone, ServerlessSpec
pc= Pinecone(api_key=apikey)

In [3]:
pc.list_indexes()
pc.delete_index("quickstart")
pc.list_indexes()

[]

In [4]:
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [5]:
#pc.delete_index(index_name)
pc.list_indexes()

[
    {
        "name": "quickstart",
        "dimension": 384,
        "metric": "cosine",
        "host": "quickstart-2s9kc7h.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

In [6]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
imdb_df = pd.read_csv('/content/drive/MyDrive/Untitled folder/imdb_movies.csv')
imdb_df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


In [8]:
imdb_df.shape

(10178, 12)

In [9]:
imdb_df.isna().sum()
imdb_df.reset_index(drop=True)
#imdb_df[imdb_df['date_x'] == ]
imdb_df.shape
imdb_df.iloc[[10051]]


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
10051,Cabin Fever,02/12/2016,47.0,Horror,"In this grisly remake of the 2002 horror hit, ...","Eli Roth, , Teresa Decher, Emily, Samuel Davis...",Cabin Fever,Released,English,1500000.0,30351664.0,US


In [10]:
imdb_df.dropna(inplace=True)

In [11]:
imdb_df['context2vectorize']= 'Name:'+ imdb_df['names'] + '. Genre '+ imdb_df['genre'] + '. ReleaseDate: '+ imdb_df['date_x']+ '. Overview:'+ imdb_df['overview'] + '. Crew: '+ imdb_df['crew']

In [12]:
imdb_df['context2vectorize'].head()

,context2vectorize
0,"Name:Creed III. Genre Drama, Action. ReleaseDa..."
1,Name:Avatar: The Way of Water. Genre Science F...
2,Name:The Super Mario Bros. Movie. Genre Animat...
3,"Name:Mummies. Genre Animation, Comedy, Family,..."
4,Name:Supercell. Genre Action. ReleaseDate: 03/...


In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
imdb_df['vector'] = imdb_df['context2vectorize'].apply(lambda sentence: model.encode(sentence))

In [33]:
imdb_df['id'] = imdb_df.index.astype(str)
#imdb_df['vector']= imdb_df['vector'].tolist()
imdb_df['values'] = imdb_df['vector'].tolist()

In [42]:
#imdb_df['metadata'] = dict(zip(imdb_df['name'], imdb_df['genre'],imdb_df['date_x'],imdb_df['overview'],imdb_df['crew']))
imdb_df['metadata'] = imdb_df.to_dict('records')
colscover = ['names','genre','date_x','crew']
imdb_df['metadata_1'] = imdb_df[colscover].to_dict('records')

In [17]:
# # Convert DataFrame to dictionary using dict() and zip() methods
# df2 = dict([(i,[x,y,z ]) for i, x,y,z in zip(df.Courses, df.Fee,df.Duration,df.Discount)])
# print("After converting a DataFrame to dictionary:\n", df2)

# # Using dict() and zip() methods
# df2 = dict([(i,[x,y,z]) for i,x,y,z in zip(df['Courses'], df['Fee'],df['Duration'],df['Discount'])])
# print("After converting a DataFrame to dictionary:\n", df2)

# imdb_df['metadata_1'] = dict([(('name':name),('name':name),('name':name),('name':name)) for name, genre,date_x,crew in zip(imdb_df['name'], imdb_df['genre'],imdb_df['date_x'],imdb_df['crew'])])

In [43]:
imdb_df['metadata']

,metadata
0,"{'names': 'Creed III', 'date_x': '03/02/2023 '..."
1,"{'names': 'Avatar: The Way of Water', 'date_x'..."
2,"{'names': 'The Super Mario Bros. Movie', 'date..."
3,"{'names': 'Mummies', 'date_x': '01/05/2023 ', ..."
4,"{'names': 'Supercell', 'date_x': '03/17/2023 '..."
...,...
10173,"{'names': '20th Century Women', 'date_x': '12/..."
10174,{'names': 'Delta Force 2: The Colombian Connec...
10175,"{'names': 'The Russia House', 'date_x': '12/21..."
10176,"{'names': 'Darkman II: The Return of Durant', ..."


In [40]:
imdb_df['metadata_1'].to_dict()

{0: {'names': 'Creed III',
  'genre': 'Drama,\xa0Action',
  'date_x': '03/02/2023 ',
  'crew': "Michael B. Jordan, Adonis Creed, Tessa Thompson, Bianca Taylor, Jonathan Majors, Damien Anderson, Wood Harris, Tony 'Little Duke' Evers, Phylicia Rashād, Mary Anne Creed, Mila Davis-Kent, Amara Creed, Florian Munteanu, Viktor Drago, José Benavidez Jr., Felix Chavez, Selenis Leyva, Laura Chavez"},
 1: {'names': 'Avatar: The Way of Water',
  'genre': 'Science Fiction,\xa0Adventure,\xa0Action',
  'date_x': '12/15/2022 ',
  'crew': "Sam Worthington, Jake Sully, Zoe Saldaña, Neytiri, Sigourney Weaver, Kiri / Dr. Grace Augustine, Stephen Lang, Colonel Miles Quaritch, Kate Winslet, Ronal, Cliff Curtis, Tonowari, Joel David Moore, Norm Spellman, CCH Pounder, Mo'at, Edie Falco, General Frances Ardmore"},
 2: {'names': 'The Super Mario Bros. Movie',
  'genre': 'Animation,\xa0Adventure,\xa0Family,\xa0Fantasy,\xa0Comedy',
  'date_x': '04/05/2023 ',
  'crew': 'Chris Pratt, Mario (voice), Anya Taylor-Joy,

In [29]:
imdb_df['vector']

,vector
0,"[-0.040763196, -0.018347144, -0.09113527, 0.00..."
1,"[-0.0940801, -0.02811879, -0.010403788, 0.0242..."
2,"[-0.11363699, -0.08523491, -0.001013042, -0.02..."
3,"[-0.038748078, -0.051306788, -0.01113439, -0.0..."
4,"[-0.08471039, -0.0024165304, -0.0019498023, -0..."
...,...
10173,"[-0.0003071547, -0.052473255, -0.057015363, 0...."
10174,"[-0.05800467, -0.06660863, -0.042030714, 0.003..."
10175,"[-0.0561947, -0.09665913, -0.043218557, 0.0324..."
10176,"[-0.02580816, -0.04478736, 0.02811991, -0.0099..."


In [46]:
upsert_df = imdb_df[['id','values']]

In [47]:
upsert_df.head()

,id,values
0,0,"[-0.040763196, -0.018347144, -0.09113527, 0.00..."
1,1,"[-0.0940801, -0.02811879, -0.010403788, 0.0242..."
2,2,"[-0.11363699, -0.08523491, -0.001013042, -0.02..."
3,3,"[-0.038748078, -0.051306788, -0.01113439, -0.0..."
4,4,"[-0.08471039, -0.0024165304, -0.0019498023, -0..."


In [48]:
index = pc.Index("quickstart")
index.upsert_from_dataframe(upsert_df, namespace="quickstart")

sending upsert requests:   0%|          | 0/10052 [00:00<?, ?it/s]

{'upserted_count': 10052}

In [49]:
xc = index.query(vector = (model.encode("Name a cartoon movie that was released in 2023")).tolist(), # python list
           top_k=10, namespace="quickstart")
xc

{'matches': [{'id': '3480', 'score': 0.514005899, 'values': []},
             {'id': '1101', 'score': 0.508785, 'values': []},
             {'id': '8803', 'score': 0.508375287, 'values': []},
             {'id': '229', 'score': 0.503485501, 'values': []},
             {'id': '5220', 'score': 0.478857577, 'values': []},
             {'id': '2206', 'score': 0.474942893, 'values': []},
             {'id': '253', 'score': 0.474803209, 'values': []},
             {'id': '7839', 'score': 0.469670653, 'values': []},
             {'id': '2238', 'score': 0.464704156, 'values': []},
             {'id': '5441', 'score': 0.463146418, 'values': []}],
 'namespace': 'quickstart',
 'usage': {'read_units': 5}}

In [50]:
xc
print_df = pd.DataFrame()
temp_df = pd.DataFrame()
for ids in xc.matches:
  print(ids.id)
  temp_df= imdb_df[imdb_df['id'] == ids.id]
  print_df = pd.concat([print_df, imdb_df[imdb_df['id'] == ids.id]])

3480
1101
8803
229
5220
2206
253
7839
2238
5441


In [51]:
 print_df

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,context2vectorize,vector,id,values,metadata,metadata_1
3480,The Lego Movie 2: The Second Part,03/21/2019,67.0,"Action, Adventure, Animation, Comedy, Family",It's been five years since everything was awes...,"Chris Pratt, Emmet Brickowski / Rex Dangervest...",The Lego Movie 2: The Second Part,Released,English,99000000.0,192456290.0,AU,Name:The Lego Movie 2: The Second Part. Genre ...,"[-0.10314516, -0.04956534, 0.05157491, -0.0671...",3480,"[-0.10314516, -0.04956534, 0.05157491, -0.0671...","{'names': 'The Lego Movie 2: The Second Part',...","{'names': 'The Lego Movie 2: The Second Part',..."
1101,Rise of the Teenage Mutant Ninja Turtles: The ...,08/05/2022,75.0,"Animation, Action, Science Fiction",When a mysterious stranger arrives from the fu...,"Ben Schwartz, Leo / Janitor Man (voice), Omar ...",Rise of the Teenage Mutant Ninja Turtles: The ...,Released,English,75600000.0,454911144.6,US,Name:Rise of the Teenage Mutant Ninja Turtles:...,"[-0.15841934, -0.024857683, -0.019100567, -0.0...",1101,"[-0.15841934, -0.024857683, -0.019100567, -0.0...",{'names': 'Rise of the Teenage Mutant Ninja Tu...,{'names': 'Rise of the Teenage Mutant Ninja Tu...
8803,Turtles Forever,11/21/2009,74.0,"Animation, Science Fiction, Adventure",Turtles Forever is a made-for-tv animated movi...,"Michael Sinterniklaas, Leonardo (voice), Wayne...",Turtles Forever,Released,English,101800000.0,596912378.2,US,"Name:Turtles Forever. Genre Animation, Science...","[-0.09230678, 0.022887262, 0.0042260885, -0.03...",8803,"[-0.09230678, 0.022887262, 0.0042260885, -0.03...","{'names': 'Turtles Forever', 'date_x': '11/21/...","{'names': 'Turtles Forever', 'genre': 'Animati..."
229,Transformers: Rise of the Beasts,06/09/2023,0.0,"Action, Adventure, Science Fiction",A ‘90s globetrotting adventure that introduces...,"Anthony Ramos, Noah, Dominique Fishback, Elena...",Transformers: Rise of the Beasts,Post Production,English,200000000.0,1240261.6,US,Name:Transformers: Rise of the Beasts. Genre A...,"[-0.16142656, -0.080576904, -0.0068986146, -0....",229,"[-0.16142656, -0.080576904, -0.0068986146, -0....","{'names': 'Transformers: Rise of the Beasts', ...","{'names': 'Transformers: Rise of the Beasts', ..."
5220,Godzilla: Planet of the Monsters,11/17/2017,63.0,"Action, Science Fiction, Animation",A desperate group of refugees attempts to reco...,"Mamoru Miyano, Haruo Sakaki (voice), Takahiro ...",GODZILLA 怪獣惑星,Released,Japanese,146200000.0,367718408.4,JP,Name:Godzilla: Planet of the Monsters. Genre A...,"[-0.059219383, -0.0056360047, 0.008891504, -0....",5220,"[-0.059219383, -0.0056360047, 0.008891504, -0....","{'names': 'Godzilla: Planet of the Monsters', ...","{'names': 'Godzilla: Planet of the Monsters', ..."
2206,The Larva Island Movie,07/23/2020,72.0,"Animation, Family","Back at home, Chuck relates the island shenani...","Hong Beom-gi, , Kang Shin-hyung, , Eddy Lee, ,...",The Larva Island Movie,Released,English,115500000.0,425732545.6,US,"Name:The Larva Island Movie. Genre Animation, ...","[-0.059650272, -0.012134851, -0.07041782, -0.0...",2206,"[-0.059650272, -0.012134851, -0.07041782, -0.0...","{'names': 'The Larva Island Movie', 'date_x': ...","{'names': 'The Larva Island Movie', 'genre': '..."
253,Godzilla vs. Kong,03/24/2021,77.0,"Action, Fantasy, Science Fiction","In a time when monsters walk the Earth, humani...","Alexander Skarsgård, Nathan Lind, Millie Bobby...",Godzilla vs. Kong,Released,English,155000000.0,468043852.0,AU,"Name:Godzilla vs. Kong. Genre Action, Fantasy,...","[-0.05737513, -0.004156905, 0.04736191, 0.0227...",253,"[-0.05737513, -0.004156905, 0.04736191, 0.0227...","{'names': 'Godzilla vs. Kong', 'date_x': '03/2...","{'names': 'Godzilla vs. Kong', 'genre': 'Actio..."
7839,The Powerpuff Girls Movie,09/30/2002,66.0,"Animation, Action, Family","Based on the hit animated television series, t...","Cathy Cavadini, Blossom (voice), Tara Strong, ...",The Powerp